# RAG pomocí služby Azure AI Agent a Semantic Kernel

Tento ukázkový kód ukazuje, jak vytvořit a spravovat agenta Azure AI pro generování s podporou vyhledávání (RAG) pomocí `Azure AI Agent Service` a `Semantic Kernel`. Agent zpracovává dotazy uživatelů na základě získaného kontextu a poskytuje přesné odpovědi.


Oprava verze SQLite  
Pokud narazíte na chybu:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

Odkomentujte tento blok kódu na začátku svého notebooku:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Importování balíčků
Následující kód importuje potřebné balíčky:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# Generování s podporou vyhledávání pomocí Semantic Kernel a Azure AI Agent Service

Tento příklad ukazuje, jak použít **Azure AI Agent Service** k provádění **Generování s podporou vyhledávání (RAG)** kombinací jazykového modelu s kontextem specifickým pro danou oblast z nahraného dokumentu.

### Jak to funguje

1. **Nahrání dokumentu**: Markdown soubor (document.md) obsahující informace (cestovní pojištění společnosti Contoso) je nahrán do agentní služby.

2. **Vytvoření vektorového úložiště**: Dokument je indexován do vektorového úložiště, aby bylo možné provádět sémantické vyhledávání jeho obsahu.

3. **Konfigurace agenta**: Agent je vytvořen pomocí modelu `gpt-4o` s následujícími přísnými pokyny:
   - Odpovídat pouze na otázky na základě získaného obsahu z dokumentu.
   - Odmítnout odpovědět, pokud je otázka mimo rozsah.

4. **Integrace nástroje pro vyhledávání v souborech**: `FileSearchTool` je zaregistrován u agenta, což umožňuje modelu během inferencí vyhledávat a získávat relevantní úryvky z indexovaného dokumentu.

5. **Interakce s uživatelem**: Uživatelé mohou klást otázky. Pokud jsou v dokumentu nalezeny relevantní informace, agent vygeneruje odpověď založenou na těchto informacích.  
   Pokud ne, agent výslovně odpoví, že dokument neobsahuje dostatečné informace.


### Hlavní funkce



Ujistěte se, že nejprve spustíte `az login` pomocí Azure CLI, aby byl zajištěn správný autentizační kontext při používání `DefaultAzureCredential`. Služba Azure AI Agent Service nepoužívá klíče API.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

Bez poskytnutí konkrétního obsahu k překladu nemohu pokračovat. Prosím, vložte text, který chcete přeložit, a já vám ho přeložím do češtiny podle uvedených pravidel.



---

**Prohlášení**:  
Tento dokument byl přeložen pomocí služby pro automatický překlad [Co-op Translator](https://github.com/Azure/co-op-translator). Ačkoli se snažíme o přesnost, mějte na paměti, že automatické překlady mohou obsahovat chyby nebo nepřesnosti. Původní dokument v jeho původním jazyce by měl být považován za autoritativní zdroj. Pro důležité informace se doporučuje profesionální lidský překlad. Neodpovídáme za žádné nedorozumění nebo nesprávné interpretace vyplývající z použití tohoto překladu.
